In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm.notebook import tqdm
tqdm.pandas()

In [31]:
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [32]:
cycle_df = pd.read_csv('cycle_table.csv')
settlement_df = pd.read_csv('settlement_table.csv')

In [33]:
'''
Converting all Date to standard Datetime Format

'''

cycle_df['start_date']= cycle_df.start_date.apply(lambda x: datetime.strptime(x, '%d/%m/%y %H:%M'))
cycle_df['end_date']= cycle_df.end_date.apply(lambda x: datetime.strptime(x, '%d/%m/%y %H:%M'))
settlement_df['created_at']= settlement_df.created_at.apply(lambda x: datetime.strptime(x, '%d/%m/%y %H:%M'))

In [34]:
'''
Function to map Creation Date for Billing Cycle
'''

def get_cycle_daterange(value):
    for row in cycle_df.iterrows():
        if row[1]['start_date']<=value['created_at']<=row[1]['end_date']:
            value['start_date']=row[1]['start_date']
            value['end_date']=row[1]['end_date']
            value['billing_cycle_id']=row[1]['id']
            return value
        else:
            pass

In [35]:
# combined_df = settlement_df.progress_apply(get_cycle_daterange,axis=1)
combined_df = pd.read_csv('combined_data.csv')

In [36]:
'''
Frequency of Bill Creation on every Billing Cycle
'''

def month_wise_billcreation_count(df):
    df = df[df.status=='billcreated']
    df.billing_cycle_id.value_counts().iplot(kind='bar', xTitle='Billing Cycle ID',
                  yTitle='count', title='Bill Creation Frequency Distribution')



In [37]:
month_wise_billcreation_count(combined_df)

In [38]:
'''
Frequency of Bill Paying on every Billing Cycle

'''

def month_wise_billpaid_count(df):
    df = df[df.status=='billpaid']
    df.billing_cycle_id.value_counts().iplot(kind='bar', xTitle='Billing Cycle ID',
                  yTitle='count', title='Bill Paid Frequency Distribution')
    

In [39]:
month_wise_billpaid_count(combined_df)

In [40]:
combined_df.user_id.value_counts()[:20].iplot(kind='bar',title='Top 20 user_id interaction frequency',xTitle='userid',yTitle='count')

In [41]:
'''
Function to Find out all bill created and paid
during each payment cycle

'''

def total_bill_status_cyclewise(combined_df):
    set_of_billing_cycle_ids = set(combined_df['billing_cycle_id'].values)
    total_bill_created = []
    total_bill_paid = []
    count_bill_created = []
    count_bill_paid = []
    for cycle_id in tqdm(set_of_billing_cycle_ids):
        df = combined_df[combined_df.billing_cycle_id==cycle_id]
        created_df = df[df.status=='billcreated']
        paid_df = df[df.status=='billpaid']
        
        count_bill_created.append(len(created_df))
        count_bill_paid.append(len(paid_df))
        
        total_bill_created.append(created_df.amount.sum())
        total_bill_paid.append(paid_df.amount.sum())
        
    settlement_df = pd.DataFrame()
    settlement_df['total_bill_created']=total_bill_created
    settlement_df['total_bill_paid']=total_bill_paid
    settlement_df['bill_cycle_id']=set_of_billing_cycle_ids
    
    
    settlement_df.iplot(x='bill_cycle_id',kind='bar',y=['total_bill_paid','total_bill_created'],xTitle='Bill cycle id',yTitle='Amount',title='Billing Transactioon Details for Each Cycle')
    return settlement_df


In [42]:
total_bill_status_cyclewise(combined_df)

,total_bill_created,total_bill_paid,bill_cycle_id
0,0,14129990,77
1,28773210,65514933,78
2,81247974,109225258,79
3,104392797,119871062,80
4,126971836,174448337,81
5,149997022,227267250,82
6,236846984,308323022,83
7,272586362,338558082,84
8,329757139,436907715,85
9,390332739,523671617,86


In [43]:
def create_empty_user_information_df():
    user_information_df = pd.DataFrame()
    user_information_df['user_id']=list(set(combined_df['user_id'].values))
    user_information_df['netAmount']=0
    return user_information_df

In [44]:
def userid_wise_billingcycle_update_fn(df,user_information_df):
    paid_count = 0
    created_count = 0
    paid_amount = 0
    created_amount = 0
    
    for row in df.iterrows():
        if row[1]['status']=='billpaid':
            user_information_df.loc[user_information_df.user_id==row[1]['user_id'],'netAmount']+=row[1]['amount']
            paid_count+=1
            paid_amount+=row[1]['amount']
            
        elif row[1]['status']=='billcreated':
            user_information_df.loc[user_information_df.user_id==row[1]['user_id'],'netAmount']-=row[1]['amount']
            created_count+=1
            created_amount+=row[1]['amount']
    
    return paid_count,paid_amount,created_count,created_amount

In [45]:
set_of_billing_cycle_ids = set(combined_df['billing_cycle_id'].values)

user_information_df = create_empty_user_information_df()
paid_count_list = []
created_count_list = []

paid_amount_list = []
created_amount_list = []

for cycle_id in tqdm(set_of_billing_cycle_ids):
    df = combined_df[combined_df.billing_cycle_id==cycle_id]
    paid_count,paid_amount,created_count,created_amount=userid_wise_billingcycle_update_fn(df,user_information_df)
    
    paid_count_list.append(paid_count)
    paid_amount_list.append(paid_amount)
    created_count_list.append(created_count)
    created_amount_list.append(created_amount)
    
    print('Completed Cycle ID:{}'.format(cycle_id))
    print('Paid Count:{}  created Count:{}'.format(paid_count,created_count))
    print('paid Amount:{} created Amount:{}'.format(paid_amount,created_amount))
    print('\n')
    

result_df = pd.DataFrame()
result_df['cycle_id']=list(set_of_billing_cycle_ids)
result_df['paid_count']=paid_count_list
result_df['created_count']=created_count_list
result_df['paid_amount']=paid_amount_list
result_df['created_amount']=created_amount_list

Completed Cycle ID:77
Paid Count:66  created Count:0
paid Amount:14129990 created Amount:0


Completed Cycle ID:78
Paid Count:293  created Count:210
paid Amount:65514933 created Amount:28773210


Completed Cycle ID:79
Paid Count:597  created Count:491
paid Amount:109225258 created Amount:81247974


Completed Cycle ID:80
Paid Count:591  created Count:594
paid Amount:119871062 created Amount:104392797


Completed Cycle ID:81
Paid Count:906  created Count:881
paid Amount:174448337 created Amount:126971836


Completed Cycle ID:82
Paid Count:1237  created Count:1004
paid Amount:227267250 created Amount:149997022


Completed Cycle ID:83
Paid Count:1497  created Count:1327
paid Amount:308323022 created Amount:236846984


Completed Cycle ID:84
Paid Count:1593  created Count:1500
paid Amount:338558082 created Amount:272586362


Completed Cycle ID:85
Paid Count:1959  created Count:1811
paid Amount:436907715 created Amount:329757139


Completed Cycle ID:86
Paid Count:2211  created Count:1851
paid

In [46]:
result_df.iplot(x='cycle_id',y=['paid_amount','created_amount'],xTitle='Billing Cycle id',yTitle='Amount',title='Cycle wise Amount')

In [47]:
result_df.iplot(x='cycle_id',y=['paid_count','created_count'],xTitle='Billing Cycle id',yTitle='Amount',title='Cycle wise user_id Count')

In [60]:
user_information_df.iplot(x='user_id',kind='line',y='netAmount',xTitle='User ID', yTitle='Net Amount', title='Net Profit of Org')

In [61]:
'''
This Function generates entire Bill cyclereport for provided userid
'''

def user_id_performance_over_entire_billcycle(userid):
    df = combined_df[combined_df.user_id==userid]
    set_of_billing_cycle_ids = set(combined_df['billing_cycle_id'].values)
    
    paid_count_list = []
    created_count_list = []

    paid_amount_list = []
    created_amount_list = []
    for cycle_id in set_of_billing_cycle_ids:
        
        paid_count=0
        created_count=0
        paid_amount = 0
        created_amount = 0
        
        if len(df[df.billing_cycle_id==cycle_id]):
            for row in df[df.billing_cycle_id==cycle_id].iterrows():
                if row[1]['status']=='billpaid':
                    paid_count+=1
                    paid_amount+=row[1]['amount']

                elif row[1]['status']=='billcreated':
                    created_count+=1
                    created_amount+=row[1]['amount']

        else:
            pass
        
        paid_count_list.append(paid_count)
        created_count_list.append(created_count)
        paid_amount_list.append(paid_amount)
        created_amount_list.append(created_amount)
        
    
    result_df = pd.DataFrame()
    result_df['cycle_id']=list(set_of_billing_cycle_ids)
    result_df['paid_count']=paid_count_list
    result_df['created_count']=created_count_list
    result_df['paid_amount']=paid_amount_list
    result_df['created_amount']=created_amount_list
    
    result_df.iplot(x='cycle_id',kind='bar',y=['paid_amount','created_amount'],xTitle='Billing Cycle id',yTitle='Amount',title='Cycle wise Amount')
    result_df.iplot(x='cycle_id',kind='bar',y=['paid_count','created_count'],xTitle='Billing Cycle id',yTitle='Count',title='Cycle wise user_id Count')
    
    return result_df

In [62]:
result = user_id_performance_over_entire_billcycle(2378) # pass desired userid to see the report